In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
chromeOption = Options()
chromeOption.add_argument("Window-size=1920x1080")
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),options=chromeOption)
from selenium.webdriver.common.by import By
import pandas as pd
import bs4
import time

targetUrl = "https://www.kurly.com/search?sword=%EB%8F%BC%EC%A7%80%EA%B3%A0%EA%B8%B0"

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
try:
    driver.get(targetUrl)
except Exception as e:
    print(e)
    
htmlCode = driver.page_source
htmlCode = bs4.BeautifulSoup(htmlCode,"html.parser")

# 총 상품수 구하기
product = htmlCode.find(name='div', attrs={"class" : 'css-crqql1 eudxpx33'})
product = product.text.split(" ")
product = product[1]
product[0:-1]
productNum = int(product[0:-1]) # 총 상품수

# 페이지 하나당 물품수 : 12
pagePro = 12
# 페이지 숫자 구하기
pageNum = int(productNum/pagePro)
#나머지가 있을때 페이지 하나 추가
if(productNum%pagePro>0):
    resultPageNum = pageNum+1

rowList=[]
# 페이지 바꾸기
for i in range(1,resultPageNum+1):
    time.sleep(5)
    # 봇을 방지하기 위해서 1~1.5초정도 대기시간이 필요하다
    # 상대시간을 쓰면 1~1.5초보다 빠르게 종료될 수도 있어서 절대시간으로 써야한다.
    targetUrl = "https://www.kurly.com/search?sword=%EB%8F%BC%EC%A7%80%EA%B3%A0%EA%B8%B0&page={}".format(i)

    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
    try:
        driver.get(targetUrl)
    except Exception as e:
        print(e)
            
    htmlCode = driver.page_source

    htmlCode = bs4.BeautifulSoup(htmlCode,"html.parser")

    time.sleep(10)
    # 넘어간 페이지마다 제품과 링크 수집
    div = htmlCode.find(name="div",attrs={"class" : 'css-11kh0cw ef36txc5'})
    aTag = div.findAll(name="a",attrs={"class" : 'css-1xyd46f e1c07x4814'})
    
    # 페이지가 넘어갈때마다 리스트를 초기화
    nameList=[]
    linkList=[]    
    
    # 데이터를 넣는 과정
    for j in range(0,len(aTag)):
        nameList.append(aTag[j].text)
        linkList.append(aTag[j].attrs["href"])
        rowList.append([nameList[j],linkList[j]])

# 제일 마지막 페이지는 끝이라는 표시가 있을 거다 .
# if를 써서 해당 값을 만나면 break;하게 만들면 끝까지 돈다.

result = pd.DataFrame(rowList)
result.columns=("제품","링크")
# 좌측정렬
result.style.set_properties(**{'text-align': 'left'})

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 58.1MB/s]


,제품,링크
0,"샛별배송[Sunterra] 보리먹인 캐나다 냉장 목심500g (에어프라이어)100g 당 1,998원10%8,991원9,990원후기380Kurly Only",/goods/1000107746
1,"샛별배송[KF365] 한돈 고추장양념 돼지불고기 1kg (냉장)100g 당 가격: 1,290원12,900원후기9,999+Kurly Only",/goods/5085769
2,"샛별배송[KF365] 한돈 앞다리 찌개용 600g (냉장)찌개용 (100g당 판매가 : 1,450원)8,700원후기9,999+Kurly Only",/goods/5056509
3,"샛별배송[8도포차] 대구식 연탄 돼지불고기은은한 불맛이 매력적인 불고기6,900원후기999+",/goods/5049219
4,"샛별배송무항생제 1등급 한돈 앞다리 3종100g 당 판매가 :1,833원5,500원후기9,999+Kurly Only",/goods/5029467
5,"샛별배송[KF365] 한돈 간장양념 돼지불고기 1kg (냉장)100g 당 가격: 1,290원12,900원후기999+Kurly Only",/goods/5085770
6,"샛별배송[KF365] 한돈 다짐육 600g (냉장)100g당 판매가:998원5,990원후기9,999+Kurly Only",/goods/5069066
7,"샛별배송무항생제 1등급 한돈 등심(카레용) 2종 100g당 판매가: 1,650원(600g)/ 1,800원(300g)5,400원~후기9,999+Kurly Only",/goods/1000025282
8,"샛별배송[도드람한돈] 앞다리 찌개용 300g (냉장) 100g당 판매가: 1,983원 5,950원후기9,999+",/goods/5059787
9,"샛별배송무항생제 1등급 한돈 다짐육 2종 100g 당 판매가: 1,167원(600g)/ 1,200원(300g)3,600원~후기9,999+Kurly Only",/goods/1000025276
